<a href="https://colab.research.google.com/github/Mouner9360/Mouner9360/blob/main/Assignment%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import cv2
import torch
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Check for GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Using device: {device}')


Using device: cpu


In [3]:
# Load the pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()  # Set model to evaluation mode


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 113MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [4]:
# Define the classes of interest
CLASSES = ['__background__', 'person', 'sports ball']

# Map COCO class indices to our classes
# COCO indices: person=1, sports ball=37
CLASS_MAPPING = {1: 'person', 37: 'sports ball'}


In [5]:
def draw_boxes(predictions, frame, threshold=0.8):
    """
    Draw bounding boxes and labels on the frame for persons and sports balls.

    Parameters:
    - predictions: Model predictions for the frame.
    - frame: The original image frame.
    - threshold: Confidence score threshold for displaying bounding boxes.

    Returns:
    - frame: The frame with bounding boxes and labels drawn.
    """
    # Extract boxes, labels, and scores
    boxes = predictions[0]['boxes'].detach().cpu().numpy()
    labels = predictions[0]['labels'].detach().cpu().numpy()
    scores = predictions[0]['scores'].detach().cpu().numpy()

    for idx, score in enumerate(scores):
        label_idx = labels[idx]
        if score > threshold and label_idx in CLASS_MAPPING:
            x1, y1, x2, y2 = boxes[idx]
            label = CLASS_MAPPING[label_idx]
            # Draw rectangle and label
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            cv2.putText(frame, f'{label}: {score:.2f}', (int(x1), int(y1)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return frame


In [1]:
# Open the input video
video_path = 'input_video.mp4'
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
else:
    print("Video opened successfully.")



NameError: name 'cv2' is not defined

In [ ]:
# Get video properties
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))


In [ ]:
# Define transformation
transform = transforms.Compose([
    transforms.ToTensor(),
])

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Total frames: {frame_count}")

# Process each frame
frame_number = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("\nFinished processing video.")
        break

    frame_number += 1
    print(f"Processing frame {frame_number}/{frame_count}", end='\r')

    # Convert the frame to a tensor
    image = transform(frame)
    image = image.to(device)

    # Get predictions
    with torch.no_grad():
        predictions = model([image])

    # Draw bounding boxes
    frame = draw_boxes(predictions, frame)

    # Write the frame to the output video
    out.write(frame)

    # Display the frame
    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
